<a href="https://colab.research.google.com/github/malshanihettiarachchi/DengueCasesPrediction/blob/dev_mal/DengueCasesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


import tensorflow as tf
from xgboost import XGBRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM , Dropout

from google.colab import files
data_to_load = files.upload()



Saving ds.csv to ds (2).csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(data_to_load['ds.csv']))
df.head(20)

,Year,Month,Dengue Victims,Temp 0C,Rainfall (mm),Humidity (%)
0,2016,1,2184,27,33.74,74
1,2016,2,1569,28,33.92,73
2,2016,3,1016,28,41.27,74
3,2016,4,942,29,101.74,77
4,2016,5,693,28,655.50,83
5,2016,6,1459,27,351.98,86
6,2016,7,3191,27,191.54,85
7,2016,8,1478,27,201.60,85
8,2016,9,939,26,88.61,85
9,2016,10,459,27,55.06,79


In [ ]:
X = df.drop(columns=["Dengue Victims"])
X.head()
y = df['Dengue Victims'].values
y[0:5]

array([2184, 1569, 1016,  942,  693])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
#Tuning Hyper-parameters for XGBoost
xgb_params = {
    "n_estimators" : [10,50,100,200,500],
    "learning_rate" : [0.001,0.01,0.1,0.05],
    "max_depth" : [3,5,7],
    "objective" : ["reg:squarederror"]
}
xgb_grid = GridSearchCV(XGBRegressor(),xgb_params,cv = 5,verbose = 1, scoring = "neg_mean_squared_error")
xgb_grid.fit(X_train,y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    6.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.05],
                         'max_depth': [3, 5, 7],
               

In [ ]:


for lookback in range(1,1):
    max_lookback = lookback
    pred_interval = 1
    input_len = max_lookback - pred_interval
    X_train =[]
    y_train = [] 
    for i in range(len(train)-max_lookback):
        X_train.append(train[i:i+max_lookback-pred_interval,2])
        y_train.append(train[i+max_lookback,2])
    X_train, y_train = np.array(X_train),np.array(y_train)
    inputs = df.iloc[len(df) - len(test) - max_lookback:,2]
    X_test = []
    y_test = []
    for i in range(len(inputs) - max_lookback):
        X_test.append(inputs[i:i+max_lookback-pred_interval])
        y_test.append(inputs[i+max_lookback])
    X_test = np.array(X_test)
    xgb = XGBRegressor(learning_rate= 0.05, max_depth= 3, n_estimators= 50,
                      objective = "reg:squarederror",verbosity= 0)
    xgb.fit(X_train,y_train)
    pred = xgb.predict(X_test)
    vis = pd.DataFrame([y_test,pred])
    vis = vis.T
    error = y_test - pred
    vis.columns = ["Actual","Predicted"]
    vis.index = test.index
    vis.plot()
    errors[max_lookback] = np.square(error).mean() #Mean Squared Error

In [ ]:
max_lookback = 1
pred_interval = 1 
input_len = max_lookback - pred_interval
X_train =[]
y_train = [] 
for i in range(len(X)-max_lookback):

    X_train.append(X.iloc[i:i+max_lookback-pred_interval,2])
    y_train.append(X.iloc[i+max_lookback,2])
X_train, y_train = np.array(X_train),np.array(y_train)
inputs = df.iloc[len(df) - len(y) - max_lookback:,2]
X_test = []
y_test = []
for i in range(len(inputs) - max_lookback):
    X_test.append(inputs[i:i+max_lookback-pred_interval])
    y_test.append(inputs[i+max_lookback])
X_test = np.array(X_test)
xgb = XGBRegressor(learning_rate= 0.05, max_depth= 3, n_estimators= 50,
                  objective = "reg:squarederror",verbosity= 0)
xgb.fit(X_train,y_train)
pred = xgb.predict(X_train)
pred_test = xgb.predict(X_test)
df2 = pd.DataFrame(pred,index = X.iloc[max_lookback:].index,
                  columns = ["xgb"])
df2 = pd.concat([df2,X],axis =1,sort = True)
df2.loc[df2["xgb"].isna(),"xgb"] = df2[df2["xgb"].isna()]["dengue"]
df2.index = pd.to_datetime(df2.index)
sc2 = MinMaxScaler()
train1 = sc2.fit_transform(df2)

XGBoostError: ignored